In [1]:
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
data_dir = pathlib.Path('/home/ravenoak/Projects/github.com/ravenoak/card-identifier/data/images/dataset')
img_height = 256
img_width = 256
img_size = (img_height, img_width),
img_channels = 3
input_shape = (img_height, img_width, img_channels)
batch_size = 32
train_seed = 1312
val_seed = 1312
train_epochs = 10
num_classes = 116

In [3]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, img_channels), include_top=False, weights="imagenet")
base_model.trainable = False

2021-10-24 12:46:26.768628: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    brightness_range=(-0.3, 0.7),
    channel_shift_range=0.1,
    rotation_range=20,
    shear_range=0.5,
    zoom_range=0.1,
    width_shift_range=0.5,
    height_shift_range=0.5,
    horizontal_flip=True,
    vertical_flip=True,
)


In [5]:
#train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#    data_dir,
#    validation_split=0.2,
#    subset="training",
#    seed=train_seed,
#    image_size=(img_height, img_width),
#    batch_size=batch_size)

train_ds = tf.data.Dataset.from_generator(
    lambda: img_gen.flow_from_directory(
        data_dir,
        batch_size=batch_size,
    ),
    output_signature=(
        tf.TensorSpec(
            shape=(None, img_height, img_width, img_channels),
            dtype=tf.int32),
        tf.TensorSpec(
            shape=(None, num_classes),
            dtype=tf.int32)
    )
)

#train_ds = train_ds.padded_batch(batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=val_seed,
    image_size=(img_height, img_width),
    batch_size=batch_size)

#val_ds

Found 58000 files belonging to 116 classes.
Using 11600 files for validation.


In [6]:
model = tf.keras.Sequential([base_model,
                             tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(num_classes, activation="softmax")
                             ])
base_learning_rate = 0.00001
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 8, 8, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 116)               148596    
Total params: 2,406,580
Trainable params: 148,596
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
history = model.fit(
    train_ds,
    epochs=train_epochs,
    validation_data=val_ds,
    batch_size=batch_size,
)

Epoch 1/10
Found 58000 images belonging to 116 classes.
   1017/Unknown - 1370s 1s/step - loss: 5.0910 - accuracy: 0.0096

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(500)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()